# Battle of Neighborhoods

### Finding best location to open a Pizza Restaurant in BRONX Borough in New York City

#### Importing required packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Step 1: Getting and exploring the data

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0] #Exploring the features

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Step 2: Creating dataframe using pandas -- data transformation

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

##### Inserting Data (such as Borough, Lat, Long values)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Step 3: Creating and exploring Bronx borough data (choosing from 5 boroughs)

In [9]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


##### Getting Long-Lat values for Bronx map

In [10]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="bronx_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.85048545, -73.8404035580209.


##### Visualizing Bronx borough in NY using Folium

In [11]:
# create map of Manhattan using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

#### Step 4: Exploring neighborhood venues using FourSquare API

##### Defining foursquare api credentials

In [12]:
CLIENT_ID = 'OO1NC33ZGUCOMYYF1LJHCBJWBV5KWA0IZLXHXMATD1LGT2Q4' # your Foursquare ID
CLIENT_SECRET = '0ADBPSMRWVE21I1J2ZYBIWS2G0FBVIC4IXT4HY1REUUPLHIB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OO1NC33ZGUCOMYYF1LJHCBJWBV5KWA0IZLXHXMATD1LGT2Q4
CLIENT_SECRET:0ADBPSMRWVE21I1J2ZYBIWS2G0FBVIC4IXT4HY1REUUPLHIB


##### Getting venue data with respect to neighborhoods

In [13]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


##### Shape & Size of Venues

In [15]:
print(bronx_venues.shape)
bronx_venues.head()

(1263, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop


#### Step 5: Analzing each neighborhood

In [16]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,Airport Tram,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Office,Optical Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
#Shape of the dataframe
bronx_onehot.shape

(1263, 174)

##### Grouping neighborhoods

In [20]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,Airport Tram,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Office,Optical Shop,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Allerton,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.032258,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.032258,0.0,0.0,0.000000,0.0,0.0,0.032258,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.064516,0.0,0.000000,0.0,0.0,0.0,0.000000,0.032258,0.0,0.0,0.064516,0.032258,0.032258,0.000000,0.032258,0.0,0.0,0.032258,0.000000,0.000000,0.032258,0.0,0.0,0.032258,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.032258,0.0,0.0,0.032258,0.0,0.000000,0.0,0.032258,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.032258,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.032258,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.064516,0.000000,0.129032,0.0,0.032258,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.064516,0.032258,0.0,0.0,0.0,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Baychester,0.0,0.0,0.0,0.045455,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.000000,0.045455,0.000000,0.045455,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,0.000000,0.0,0.0,0.045455,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.045455,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.045455,0.0,0.0,0.045455,0.000000,0.000000,0.045455,0.0,0.0,0.045455,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.045455,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.

In [21]:
#Shape of the dataframe
bronx_grouped.shape

(52, 174)

#### Step 6: Finding the top 3 frequently visited venues

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allerton,Pizza Place,Supermarket,Deli / Bodega
1,Baychester,Breakfast Spot,Donut Shop,Sandwich Place
2,Bedford Park,Diner,Pizza Place,Deli / Bodega
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega
4,Bronxdale,Italian Restaurant,Park,Eastern European Restaurant


#### Step 7: Neighborhood Clustering using K-Means

##### Cluster training

In [25]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 1, 1, 2, 2, 4, 2], dtype=int32)

##### Adding cluster labels to the dataframe

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Pharmacy,Sandwich Place,Ice Cream Shop
1,Bronx,Co-op City,40.874294,-73.829939,2,Baseball Field,Restaurant,Bus Station
2,Bronx,Eastchester,40.887556,-73.827806,2,Caribbean Restaurant,Bus Station,Deli / Bodega
3,Bronx,Fieldston,40.895437,-73.905643,0,River,High School,Playground
4,Bronx,Riverdale,40.890834,-73.912585,1,Park,Bus Station,Home Service


##### Visualizing clusters based on venue data using Folium

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Step 8: Examining Clusters

##### Cluster 0

In [37]:
bronx_clust_0 = bronx_merged.loc[
    bronx_merged['Cluster Labels'] == 0, 
    bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]
]
popular_venues_c0 = bronx_clust_0.groupby(['1st Most Common Venue']).size()
print('RESULT')
print('Most common venue in cluster 0: ', popular_venues_c0[popular_venues_c0 == popular_venues_c0.max()].index[0])
print()
bronx_clust_0

RESULT
Most common venue in cluster 0:  River



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
3,Fieldston,River,High School,Playground


##### Cluster 1

In [41]:
bronx_clust_1 = bronx_merged.loc[
    bronx_merged['Cluster Labels'] == 1, 
    bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]
]
popular_venues_c1 = bronx_clust_1.groupby(['1st Most Common Venue']).size()
print('RESULT')
print('Most common venue in cluster 1: ', popular_venues_c1[popular_venues_c1 == popular_venues_c1.max()].index[0])
print()
bronx_clust_1

RESULT
Most common venue in cluster 1:  Pizza Place



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
4,Riverdale,Park,Bus Station,Home Service
7,Norwood,Pizza Place,Deli / Bodega,Park
27,Throgs Neck,Deli / Bodega,Pizza Place,Coffee Shop
31,Van Nest,Deli / Bodega,Pizza Place,Supermarket
32,Morris Park,Pizza Place,Burger Joint,Bakery
33,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega
35,North Riverdale,Pizza Place,Italian Restaurant,Coffee Shop
38,Edgewater Park,Italian Restaurant,Deli / Bodega,Pizza Place
39,Castle Hill,Pizza Place,Pharmacy,Market
49,Bronxdale,Italian Restaurant,Park,Eastern European Restaurant


##### Cluster 2

In [38]:
bronx_clust_2 = bronx_merged.loc[
    bronx_merged['Cluster Labels'] == 2, 
    bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]
]
popular_venues_c2 = bronx_clust_2.groupby(['1st Most Common Venue']).size()
print('RESULT')
print('Most common venue in cluster 2: ', popular_venues_c2[popular_venues_c2 == popular_venues_c2.max()].index[0])
print()
bronx_clust_2

RESULT
Most common venue in cluster 2:  Pizza Place



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Wakefield,Pharmacy,Sandwich Place,Ice Cream Shop
1,Co-op City,Baseball Field,Restaurant,Bus Station
2,Eastchester,Caribbean Restaurant,Bus Station,Deli / Bodega
5,Kingsbridge,Pizza Place,Sandwich Place,Supermarket
6,Woodlawn,Pub,Playground,Deli / Bodega
9,Baychester,Breakfast Spot,Donut Shop,Sandwich Place
10,Pelham Parkway,Pizza Place,Italian Restaurant,Bus Station
11,City Island,Harbor / Marina,Thrift / Vintage Store,Ice Cream Shop
12,Bedford Park,Diner,Pizza Place,Deli / Bodega
13,University Heights,Pizza Place,Food,Bakery


##### Cluster 3

In [39]:
bronx_clust_3 = bronx_merged.loc[
    bronx_merged['Cluster Labels'] == 3, 
    bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]
]
popular_venues_c3 = bronx_clust_3.groupby(['1st Most Common Venue']).size()
print('RESULT')
print('Most common venue in cluster 3: ', popular_venues_c3[popular_venues_c3 == popular_venues_c3.max()].index[0])
print()
bronx_clust_3

RESULT
Most common venue in cluster 3:  Soup Place



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
8,Williamsbridge,Soup Place,Bar,Caribbean Restaurant


##### Cluster 4

In [40]:
bronx_clust_4 = bronx_merged.loc[
    bronx_merged['Cluster Labels'] == 4, 
    bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]
]
popular_venues_c4 = bronx_clust_4.groupby(['1st Most Common Venue']).size()
print('RESULT')
print('Most common venue in cluster 4: ', popular_venues_c4[popular_venues_c4 == popular_venues_c4.max()].index[0])
print()
bronx_clust_4

RESULT
Most common venue in cluster 4:  Park



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
26,Clason Point,Park,South American Restaurant,Boat or Ferry
34,Spuyten Duyvil,Park,Waste Facility,Pharmacy


#### Final Step: Recommendation based on observation

##### Retrieving neighborhoods of Clusters 1 & 2 having Pizza Place as most popular

In [59]:
#Cluster 1 neighborhoods
clust1 = bronx_clust_1.loc[bronx_clust_1['1st Most Common Venue'] == 'Pizza Place', 'Neighborhood']
#Cluster 2 neighborhoods
clust2 = bronx_clust_2.loc[bronx_clust_2['1st Most Common Venue'] == 'Pizza Place', 'Neighborhood']

#Converting to DataFrames
clust1 = pd.DataFrame(clust1)
clust2 = pd.DataFrame(clust2)

#Merging two dfs
result_df = pd.concat([clust1, clust2])
result_df.reset_index(drop=True)

,Neighborhood
0,Norwood
1,Morris Park
2,North Riverdale
3,Castle Hill
4,Kingsbridge
5,Pelham Parkway
6,University Heights
7,Fordham
8,East Tremont
9,High Bridge


<p><b> The above 16 neighborhoods are recommended places to open a Pizza Restaurant in the Bronx borough of New York city.</b></p> 